# Problematic

1) what are the most popular restaurant in Geneva
2) how many cult places per postal code
3) how many police station per postal code
4) What is the postal code with the lowestnumber of recycling bins



## Exploration phase


In [1]:
import xml.etree.cElementTree as ET
import pprint as pp
import re
import codecs
import json
import pymongo
import collections as col
from itertools import islice

In [2]:
OSM_FILE = "Geneva_MAP.osm"

In [3]:

SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### COUNT THE DIFFERENT TAGS

In [4]:
tags = {}
for event,elem in ET.iterparse(SAMPLE_FILE):
    if elem.tag in tags:
        tags[elem.tag] +=1
    else:
        tags[elem.tag]=1
print tags

{'node': 237342, 'nd': 334544, 'member': 3796, 'tag': 123248, 'relation': 334, 'way': 36204, 'osm': 1}


In [5]:
tags_details = {}
keys = ["amenity","building","emergency","historic","military","office","shop","sport"]

def create_tags_details(binder, list_keys, filename):
    """
    Create a dictionnary of every attributes value of the list of attributes key named "keys".
    This function aims to help me understand what's inside the datased and what type of analysis could be made.
    """
    for key in list_keys:
        binder[key] = {}
    for event, elem in ET.iterparse(filename, events = ("start",)):
        if elem.tag == "tag":
            for tag in elem.iter("tag"):
                for key in list_keys:
                    if elem.attrib["k"] == key:
                        if tag.attrib["v"] not in binder[key]:
                            binder[key][tag.attrib["v"]] = 1
                        else:    
                            binder[key][tag.attrib["v"]] += 1
    return binder

In [6]:
create_tags_details(tags_details, keys, SAMPLE_FILE)

{'amenity': {'Music_venue': 1,
  'animal_boarding': 2,
  'arts_centre': 1,
  'atm': 7,
  'bank': 8,
  'bar': 8,
  'bench': 95,
  'bench;waste_basket': 1,
  'bicycle_parking': 21,
  'brothel': 1,
  'cafe': 14,
  'car_rental': 1,
  'car_sharing': 7,
  'car_wash': 3,
  'charging_station': 1,
  'childcare': 1,
  'cinema': 1,
  'clock': 2,
  'college': 2,
  'community_centre': 4,
  'concert_hall': 1,
  'courthouse': 1,
  'customs': 2,
  'dentist': 1,
  'doctors': 2,
  'drinking_water': 21,
  'driving_school': 1,
  'embassy': 5,
  'events_centre': 1,
  'fast_food': 11,
  'ferry_terminal': 1,
  'fire_station': 4,
  'fountain': 19,
  'fuel': 13,
  'hospital': 9,
  'ice_cream': 1,
  'kindergarten': 9,
  'library': 3,
  'motorcycle_parking': 5,
  'nightclub': 3,
  'parking': 365,
  'parking_entrance': 11,
  'parking_space': 9,
  'pharmacy': 15,
  'place_of_worship': 33,
  'police': 6,
  'post_box': 16,
  'post_office': 9,
  'prison': 1,
  'pub': 1,
  'public_bookcase': 1,
  'public_building': 9,

1) what are the most popular kind of restaurant in Geneva
2) how many cult places per postal code
3) What is the safest place in Geneva (how many police+doctors, pharmacy, fire station  per postal code)
4) How many parking per postal code? cross it with demographic data
5) What is the postal code with the lowestnumber of recycling bins
6) what are the most popular kind of sports in Geneva
7) club versus retirement home per postal code
8) Theatre versus cinema per postal code




### Characters analysis 

Analysis of the k value for each "tag"


In [7]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        att = element.attrib['k']
        lo = lower.search(att)
        loco = lower_colon.search(att)
        pb = problemchars.search(att)
        if lo:
            keys['lower']+=1
        elif loco:
            keys['lower_colon']+=1
        elif pb:
            keys['problemchars']+=1
        else:
            keys['other']+=1
            
        pass
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [8]:
process_map("sample.osm")

{'lower': 102133, 'lower_colon': 20736, 'other': 379, 'problemchars': 0}

In [9]:
for _, element in ET.iterparse("sample.osm"):
    if element.tag == "tag":
        att = element.attrib['k']
        lo = lower.search(att)
        loco = lower_colon.search(att)
        pb = problemchars.search(att)
        if lo:
            pass
        elif loco:
            pass
        elif pb:
            pass
        else:
            print(element.attrib['k'])


        pass
        


ref:INSEE
ref:SIREN
ref:INSEE
ref:SIREN
ref:INSEE
ref:FR:SIREN
openGeoDB:loc_id
openGeoDB:community_identification_number
openGeoDB:name
openGeoDB:layer
openGeoDB:loc_id
openGeoDB:is_in_loc_id
openGeoDB:community_identification_number
openGeoDB:layer
openGeoDB:loc_id
openGeoDB:is_in_loc_id
openGeoDB:community_identification_number
ref:INSEE
ref:FR:SIREN
openGeoDB:loc_id
openGeoDB:community_identification_number
openGeoDB:name
openGeoDB:type
openGeoDB:is_in
openGeoDB:layer
openGeoDB:loc_id
openGeoDB:version
openGeoDB:sort_name
openGeoDB:population
openGeoDB:auto_update
openGeoDB:is_in_loc_id
openGeoDB:postal_codes
openGeoDB:license_plate_code
openGeoDB:community_identification_number
openGeoDB:layer
openGeoDB:loc_id
openGeoDB:is_in_loc_id
openGeoDB:license_plate_code
openGeoDB:community_identification_number
openGeoDB:is_in
openGeoDB:layer
openGeoDB:loc_id
openGeoDB:is_in_loc_id
openGeoDB:community_identification_number
openGeoDB:name
openGeoDB:type
openGeoDB:is_in
openGeoDB:layer
openG

#### Auditing and correcting street names



In [28]:
# create a dictionnary to store the street types
street_types = col.defaultdict(set)
# Regular expression to isolate non expeced street names
street_type_re = re.compile(r'^\w+', re.IGNORECASE)
#list of expected street names:
expected_street_type = ["Rue","All","Parc", "Passage", "Quai",  "Chemin", "Route", "Ruelle", "Cours", "Avenue", "Impasse", "Mail","Boulevard", "Square", "Place", "Allee"]




In [29]:
def audit_street_map_2(file_in, street_types, pretty = False):
    for _, element in ET.iterparse(file_in):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == "addr:street":
                    m = street_type_re.search(tag.attrib['v'])
                    if m:
                        street_type = m.group()
                        if street_type not in expected_street_type:
                            street_types[street_type].add(tag.attrib['v'])
    pp.pprint(dict(street_types))

In [30]:
audit_street_map_2("sample.osm",street_types,True)

{'Bernex': set(['Bernex en Combes']),
 'Centre': set(['Centre village']),
 'Champ': set(['Champ Corboz']),
 u'Cit': set([u'Cit\xe9 Vieusseux', u'Cit\xe9 Villars']),
 'Clos': set(['Clos Mallet-Du-Pan',
              'Clos des Anges',
              'Clos des Ecornaches',
              'Clos des Vignes',
              'Clos-de-la-Fonderie']),
 'Cour': set(['Cour de Saint-Pierre']),
 'Galerie': set(['Galerie Jean-Malbuisson']),
 'Grand': set(["Grand'Rue", 'Grand-Montfleury', 'Grand-Rue']),
 'Grande': set(['Grande Rue']),
 u'Hameau': set(['Hameau de Fossard', u'Hameau des Ch\xeanes']),
 'L': set(["L'Ancienne-Route"]),
 'La': set(['La Voie-de-Gex',
            u'La Voie-de-Mo\xebns',
            'La Voie-du-Coin',
            'La Vy-Verte',
            'La Vy-la-Tour',
            'La-Renfile']),
 'Le': set(['Le Mail', u'Le Prieur\xe9']),
 'Les': set(['Les Hauts de Magny', 'Les Narcisses']),
 u'Lotissement': set([u'Lotissement Le Clos des Vall\xe9es',
                      'Lotissement Les V

In [31]:
for _, element in ET.iterparse("sample.osm"):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == "addr:street":
                    if tag.attrib['v'].startswith("chemin"):
                        tag.attrib['v'] = re.sub(r'^chemin', 'Chemin', problem)
                        pp.pprint(tag.attrib['v'])
                    

'Chemin des Bois'


In [32]:
audit_street_map_2("sample.osm",street_types,True)

{'Bernex': set(['Bernex en Combes']),
 'Centre': set(['Centre village']),
 'Champ': set(['Champ Corboz']),
 u'Cit': set([u'Cit\xe9 Vieusseux', u'Cit\xe9 Villars']),
 'Clos': set(['Clos Mallet-Du-Pan',
              'Clos des Anges',
              'Clos des Ecornaches',
              'Clos des Vignes',
              'Clos-de-la-Fonderie']),
 'Cour': set(['Cour de Saint-Pierre']),
 'Galerie': set(['Galerie Jean-Malbuisson']),
 'Grand': set(["Grand'Rue", 'Grand-Montfleury', 'Grand-Rue']),
 'Grande': set(['Grande Rue']),
 u'Hameau': set(['Hameau de Fossard', u'Hameau des Ch\xeanes']),
 'L': set(["L'Ancienne-Route"]),
 'La': set(['La Voie-de-Gex',
            u'La Voie-de-Mo\xebns',
            'La Voie-du-Coin',
            'La Vy-Verte',
            'La Vy-la-Tour',
            'La-Renfile']),
 'Le': set(['Le Mail', u'Le Prieur\xe9']),
 'Les': set(['Les Hauts de Magny', 'Les Narcisses']),
 u'Lotissement': set([u'Lotissement Le Clos des Vall\xe9es',
                      'Lotissement Les V

Some cleaning needs to be done: <br>
Data now : replacement<br>
chemin : Chemin<br>
pl : Place<br>
la : La<br>
rue : Rue<br>


In [22]:
def clean_street_type_2(address_tag,address_dict):
    key = re.sub('addr:', '', address_tag['k']).strip()
    if address_tag['v'].startswith("chemin"):
        address_dict[key] = re.sub(r'^chemin', 'Chemin', address_tag['v'])
    elif address_tag['v'].startswith("pl"):
        address_dict[key] = re.sub(r'^pl', 'Place', address_tag['v'])
    elif address_tag['v'].startswith("la"):
        address_dict[key] = re.sub(r'^la', 'La', address_tag['v'])
    elif address_tag['v'].startswith("rue"):
        address_dict[key] = re.sub(r'^rue', 'Rue', address_tag['v'])        
    else:
        address_dict[key] = address_tag['v']

In [15]:

#check addr: with :
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')

def clean_street_type(address_tag,address_dict):
    """
    This function aims to transform address tags and add them as a key / value pair of a dedicated address dictionary. 
    The function will take the address tag and the address dictionary as arguments:
        address_tag: every sub tag of a "node" or "way" top level tag.
        address_dict: a dictionnary where we store key / value pair of adress elements (postcode, street, etc..)
    The function first creates a cleaned key based on the current key of the tag by removing the "addr:" part of the string.
    If after removing "addr:" the function still detects a colon, the tag isn't added to the dictionary because the data entry was not clean enought.
    Else, if the value of the tag starts with "AVE" as first word, "AVE" is replaced by "Avenue".
    Finally, the key / value is added to the address dictionary.
    """
    key = re.sub('addr:', '', address_tag['k']).strip()
    if lower_colon.match(key):
        return None
    else:
        if address_tag['v'].startswith("chemin"):
            address_dict[key] = re.sub(r'^chemin', 'Chemin', address_tag['v'])
        elif address_tag['v'].startswith("pl"):
            address_dict[key] = re.sub(r'^pl', 'Place', address_tag['v'])
        elif address_tag['v'].startswith("la"):
            address_dict[key] = re.sub(r'^la', 'La', address_tag['v'])
        elif address_tag['v'].startswith("rue"):
            address_dict[key] = re.sub(r'^rue', 'Rue', address_tag['v'])        
        else:
            address_dict[key] = address_tag['v']

split the above function in 2

In [ ]:
tree = ET.parse(OSM_FILE)
root = tree.getroot()
test_tags = root.findall("./node/tag")

test_address_tags_dict = {}

for tag in test_tags:
    tag_dict = {}
    if tag.attrib['k'].startswith("addr:"):
        clean_street_type_2(tag.attrib,tag_dict)
    if tag_dict:
        test_address_tags_dict[tag.attrib['k'] + tag.attrib['v']] = tag_dict


def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

pp.pprint(take(50, test_address_tags_dict.iteritems()))

### Auditing and correcting postal codes

In [37]:
postal_codes = set()
postal_codes_re = re.compile(r'^[1-9]\d{3}$', re.IGNORECASE)

for _, element in ET.iterparse("sample.osm"):
    if element.tag == "way" or element.tag == "node":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:postcode": 
                pp.pprint(tag.attrib['v'])
                

'74140'
'01220'
'74140'
'74160'
'74160'
'1232'
'1205'
'1205'
'1217'
'74380'
'74130'
'74930'
'1213'
'1213'
'1213'
'74160'
'1211'
'1211'
'1205'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1204'
'1205'
'1205'
'1205'
'1205'
'1205'
'1204'
'74100'
'74100'
'74100'
'74100'
'1205'
'1205'
'1205'
'1205'
'1205'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'
'1227'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'
'1201'


In [38]:
for _, element in ET.iterparse("sample.osm"):
    if element.tag == "way" or element.tag == "node":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:postcode": 
                m = postal_codes_re.search(tag.attrib['v'])
                if m == None:
                    postal_codes.add(tag.attrib['v'])
pp.pprint(postal_codes)


set(['01170',
     '01210',
     '01220',
     '01280',
     '01630',
     '01710',
     '74100',
     '74130',
     '74140',
     '74160',
     '74240',
     '74250',
     '74380',
     '74520',
     '74560',
     '74580',
     '74930'])


In [39]:
def audit_post_codes(File, postal_codes, pretty = False) :
    for _, element in ET.iterparse(File):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == "addr:postcode": 
                    m = postal_codes_re.search(tag.attrib['v'])
                    if m == None:
                        postal_codes.add(tag.attrib['v'])
    pp.pprint(postal_codes)


In [40]:
audit_post_codes("sample.osm",postal_codes, pretty = False)

set(['01170',
     '01210',
     '01220',
     '01280',
     '01630',
     '01710',
     '74100',
     '74130',
     '74140',
     '74160',
     '74240',
     '74250',
     '74380',
     '74520',
     '74560',
     '74580',
     '74930'])


This function shows that some postcodes don't follow the Swiss rule, because they seem to be on the other side of the border, in nearby France, let's check that

In [53]:
postal_codes_2 =set()
def audit_post_codes_2(File, postal_codes, pretty = False) :
    for _, element in ET.iterparse(File):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == "addr:postcode": 
                    m = postal_codes_re.search(tag.attrib['v'])
                    pcode = tag.attrib['v'] #save this postal code in a variable to be used later
                    if m == None:
                        for tag in element.iter("tag"):
                            if tag.attrib['k'] == "addr:country":
                                country = tag.attrib['v']
                                if country == "FR":
                                    None
                                else:
                                    postal_codes_2.add(pcode)
    pp.pprint(postal_codes_2)


In [54]:
audit_post_codes_2("sample.osm",postal_codes, pretty = False)

set([])


Removing the French Postal codes, we see that none of the swiss postal codes are wrongly formated 